In [4]:
import sys
sys.path.append('..')
from re import L
import torch
import torchvision
import torchvision.transforms as transforms
import clip
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import os
# from kclip import model
import sys
import torch.optim as optim
# import pytorch_model_summary
from Metrics import nmi, acc, base_kmeans_model_evaluation, kmeans_with_init
import torch
from torch.utils.tensorboard import SummaryWriter
# from adamp import AdamP
from networks import *

In [5]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [6]:
batch_size = 50
device = torch.device('cuda') if torch.cuda.is_available() else "cpu"
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 4


In [4]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                       download=True, transform=preprocess)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=preprocess)
concatset = torch.utils.data.ConcatDataset([testset,trainset])
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False)

NameError: name 'preprocess' is not defined

In [4]:
# model, preprocess = clip.load("ViT-B/32")
clip_model, preprocess = clip.load("ViT-L/14")
# clip_model = clip_model.to(device).float()
# for param in model.parameters():
#     param.require_grad = False
# clip_model.eval()
# print('done')


num_classes = 10
model = CustomCLIP(clip_model.to('cpu'), num_classes)

# for name, param in model.named_parameters():
#     if "prompt_learner" not in name:
#         param.requires_grad_(False)

# model.to(device)

Initializing class-specific contexts
Initial context: "X X X X X X X X X X X X X X X X"
Number of context words (tokens): 16


In [5]:
clip_model.to(device)

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)


In [6]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                       download=True, transform=preprocess)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=preprocess)
concatset = torch.utils.data.ConcatDataset([testset,trainset])
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
total_epoch = 50
for epoch in range(total_epoch):
    score = 0
    total_loss = 0
    print('==================================================================')
    print('epoch:', epoch)
    for i, data in enumerate(testloader, 0):
        inputs, targets = data
        inputs, targets = inputs.to(device), targets.to(device)
        output = model(inputs)
        loss = criterion(output, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        pred = torch.argmax(output, dim=1)
        score += torch.sum(pred == targets)
        total_loss += loss.item()
        if i % 50 == 49:
            print('train_iter:', i, '/', len(testloader))
    scheduler.step()

epoch: 0


TypeError: forward() missing 1 required positional argument: 'text'

In [15]:
with torch.no_grad():
    model.to(device)
    first_centroids, label, acc = base_kmeans_model_evaluation(model, testloader, 10)

ValueError: could not broadcast input array from shape (50,10) into shape (50,768)

In [33]:
print(label[:20])

[0 6 6 4 9 9 1 9 0 1 4 7 0 8 7 6 0 8 6 9]


In [34]:
print(testset.targets[:20])

[3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6]


In [35]:
for i,item in enumerate(testloader):
    data ,label_ = item
    print(label_)
    break


tensor([3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9,
        5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 2, 4, 1, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9,
        7, 6])


In [36]:
testset.targets = label
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False)
for i,item in enumerate(testloader):
    data ,label_ = item
    print(label_)
    break


tensor([0, 6, 6, 4, 9, 9, 1, 9, 0, 1, 4, 7, 0, 8, 7, 6, 0, 8, 6, 9, 8, 4, 5, 7,
        0, 2, 5, 4, 7, 9, 9, 0, 5, 0, 7, 2, 8, 1, 7, 0, 5, 9, 0, 9, 4, 7, 0, 7,
        8, 9], dtype=torch.int32)


In [10]:
classes_list =   ['a photo of amphibian'  ,'a photo of a frog','a photo of a bird of animal','a photo of a cat of animal','a photo of a deer of animal',
               'a photo of a dog','a photo of a frog of animal','a photo of a horse of animal','a photo of a ship','a photo of a truck']

In [8]:
classes_list_list = [['airplane', 'aircraft', 'craft', 'vehicle', 'conveyance', 'artifact'],
                ['car', 'motor_vehicle', 'self-propelled_vehicle', 'wheeled_vehicle', 'container', 'artifact'],
                ['bird', 'vertebrate', 'animal'],
                ['cat', 'feline', 'carnivore', 'placental', 'mammal', 'vertebrate', 'animal'],
                ['deer', 'ruminant', 'even-toed_ungulate', 'ungulate', 'placental', 'mammal', 'vertebrate', 'animal'],
                ['dog', 'domestic_animal', 'animal'],
                ['frog', 'amphibian', 'vertebrate', 'animal'],
                ['horse', 'equine', 'odd-toed_ungulate', 'ungulate', 'placental', 'mammal', 'vertebrate', 'animal'],
                ['ship', 'vessel', 'craft', 'vehicle', 'conveyance', 'artifact'],
                ['truck', 'motor_vehicle', 'self-propelled_vehicle', 'wheeled_vehicle', 'container', 'artifact']]
# classes_list = ['a photo of a '+ ' of '.join(txt) for txt in classes_list_list]
classes_list = [' of '.join(txt) for txt in classes_list_list]
classes_list = [txt.replace("_", " ") for txt in classes_list]
classes_list

['airplane of aircraft of craft of vehicle of conveyance of artifact',
 'car of motor vehicle of self-propelled vehicle of wheeled vehicle of container of artifact',
 'bird of vertebrate of animal',
 'cat of feline of carnivore of placental of mammal of vertebrate of animal',
 'deer of ruminant of even-toed ungulate of ungulate of placental of mammal of vertebrate of animal',
 'dog of domestic animal of animal',
 'frog of amphibian of vertebrate of animal',
 'horse of equine of odd-toed ungulate of ungulate of placental of mammal of vertebrate of animal',
 'ship of vessel of craft of vehicle of conveyance of artifact',
 'truck of motor vehicle of self-propelled vehicle of wheeled vehicle of container of artifact']

In [6]:
classes_list = testset.classes

In [11]:
with torch.no_grad():
    classes_list = clip.tokenize(classes_list).to(device)
    centroid_candidate_text = clip_model.encode_text(classes_list)
    centroid_candidate_text = centroid_candidate_text / centroid_candidate_text.norm(dim=-1, keepdim=True)


In [12]:
centroid_candidate_text

tensor([[-0.0040,  0.0013,  0.0406,  ...,  0.0234, -0.0158,  0.0099],
        [ 0.0134, -0.0096,  0.0328,  ...,  0.0333,  0.0104,  0.0182],
        [-0.0104,  0.0161,  0.0366,  ...,  0.0064,  0.0082,  0.0098],
        ...,
        [ 0.0255,  0.0063,  0.0246,  ...,  0.0091, -0.0214, -0.0057],
        [-0.0144, -0.0089,  0.0215,  ...,  0.0299,  0.0108, -0.0028],
        [-0.0123, -0.0028,  0.0378,  ..., -0.0095,  0.0780,  0.0179]],
       device='cuda:0', dtype=torch.float16)

In [15]:
print(centroid_candidate_text.shape)

torch.Size([10, 768])


In [10]:
## 빈도수로 label찾는 방법

testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False)

for k in range(5):
    centroids = {}
    for i in range(10):
        centroids[i] = torch.zeros((10))
        pred_centroids_text = []
    with torch.no_grad():
        if k == 0 :
            print('===============',k+1,'번째 iter===============')
            first_centroids, label, acc = base_kmeans_model_evaluation(model, testloader, 10)
            for i, (x, target) in enumerate(testloader):
                x = x.to(device)
                image_features = model.encode_image(x)
                image_features = image_features / image_features.norm(dim=-1, keepdim=True)
                similarity = image_features@centroid_candidate_text.t()
                pred_labels = similarity.argmax(dim=-1)
                b = x.shape[0]
                for j in range(b):
                    centroids[label[i].item()][pred_labels[j].item()] +=1
                for num in range(10):
                    pred_label = centroids[num].argmax(dim=-1).cpu().item()
                    pred_centroids_text.append(classes_list[pred_label])
                    pred_centroids_text = model.encode_text(pred_centroids_text)
                    pred_centroids_text = pred_centroids_text / pred_centroids_text.norm(dim=-1, keepdim=True)
        else:
            print('===============',k+1,'번째 iter===============')
            first_centroids, label , acc = kmeans_with_init(model, testloader,  10, first_centroids/2+pred_centroids_text.cpu().numpy()/2)


=============== 1 번째 iter===============
image_ACC 0.8302
image_NMI 0.8340571290757178


TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not list

In [ ]:
## softmax 앙상블로 label 찾는 방법

testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False)
softmax = torch.nn.Softmax()
for k in range(5):
    centroids = {}
    for i in range(10):
        centroids[i] = []
        pred_centroids_text = []
    with torch.no_grad():
        if k == 0 :
            print('===============',k+1,'번째 iter===============')
            first_centroids, label, acc = base_kmeans_model_evaluation(model, testloader, 10)
            for i, (x, target) in enumerate(testloader):
                x = x.to(device)
                image_features = model.encode_image(x)
                image_features = image_features / image_features.norm(dim=-1, keepdim=True)
                similarity = image_features@centroid_candidate_text.t()
#                 pred_labels = similarity.argmax(dim=-1)
                similarity = softmax(similarity)
                centroids[[label][i].item()].append(similarity)
                b = x.shape[0]
#                 for j in range(b):
#                     centroids[label[i].item()][pred_labels[j].item()] +=1

                for num in range(10):
                    pred_label = centroids[num].argmax(dim=-1).cpu().item()
                    pred_centroids_text.append(classes_list[pred_lable])
                    pred_centroids_text = model.encode_text(pred_centroids_text)
                    pred_centroids_text = pred_centroids_text / pred_centroids_text.norm(dim=-1, keepdim=True)
        else:
            print('===============',k+1,'번째 iter===============')
            first_centroids, label , acc = kmeans_with_init(model, testloader,  10, first_centroids/2+pred_centroids_text.cpu().numpy()/2)


In [33]:
testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False)
concatloader = torch.utils.data.DataLoader(concatset, batch_size=1,
                                         shuffle=False)
with torch.no_grad():
    first_centroids, label, acc = base_kmeans_model_evaluation(model, concatloader, 10)

image_ACC 0.8346666666666667
image_NMI 0.8392988775279258


In [38]:
print(len(label))

60000


In [39]:
softmax = torch.nn.Softmax(dim=-1)

centroids = [torch.zeros(1,10) for _ in range(10)]
cluster_gt_match = [[0 for _ in range(10)] for _ in range(10)]
count = [0 for i in range(10)]
with torch.no_grad():
    for i, (x, target) in enumerate(testloader):
        x = x.to(device)
        image_features = model.encode_image(x)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        similarity = image_features@centroid_candidate_text.t()
        similarity = softmax(similarity)
        centroids[label[i]] += similarity.to('cpu')
        b = x.shape[0]

In [40]:
cluster_gt_match = [[0 for _ in range(10)] for _ in range(10)]
for i, (x, target) in enumerate(testloader):
    cluster_gt_match[label[i]][target] += 1

In [41]:
## 각 cluster에 대해 gt_label의 분포
cluster_gt_match

[[0, 0, 0, 0, 6, 4, 0, 900, 0, 0],
 [58, 16, 54, 60, 110, 27, 82, 69, 40, 14],
 [3, 0, 0, 4, 3, 1, 877, 0, 0, 0],
 [3, 963, 0, 1, 1, 0, 0, 1, 5, 27],
 [2, 0, 18, 16, 865, 3, 13, 22, 1, 0],
 [1, 0, 13, 917, 7, 963, 20, 7, 3, 0],
 [5, 0, 911, 1, 8, 2, 7, 1, 0, 0],
 [17, 0, 0, 0, 0, 0, 0, 0, 934, 3],
 [3, 21, 0, 0, 0, 0, 0, 0, 3, 954],
 [908, 0, 4, 1, 0, 0, 1, 0, 14, 2]]

In [29]:
## softmax로 골랐을 때
for i in range(10):
    print(centroids[i].argmax(dim=-1).item())

3
1
2
7
0
4
8
9
7
6


In [37]:
## 각 cluster에 대해 gt_label의 분포
cluster_gt_match

[[0, 0, 0, 0, 6, 4, 0, 900, 0, 0],
 [58, 16, 54, 60, 110, 27, 82, 69, 40, 14],
 [3, 0, 0, 4, 3, 1, 877, 0, 0, 0],
 [3, 963, 0, 1, 1, 0, 0, 1, 5, 27],
 [2, 0, 18, 16, 865, 3, 13, 22, 1, 0],
 [1, 0, 13, 917, 7, 963, 20, 7, 3, 0],
 [5, 0, 911, 1, 8, 2, 7, 1, 0, 0],
 [17, 0, 0, 0, 0, 0, 0, 0, 934, 3],
 [3, 21, 0, 0, 0, 0, 0, 0, 3, 954],
 [908, 0, 4, 1, 0, 0, 1, 0, 14, 2]]

In [22]:
## softmax로 골랐을 때
for i in range(10):
    print(centroids[i].argmax(dim=-1).item())

5
1
2
4
0
4
8
9
7
6


In [45]:
print(centroids)
for i in range(10):
    print(centroids[i].cpu().argmax(dim=-1))

[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)


In [42]:
centroids

[[tensor([0.0992, 0.0986, 0.1003, 0.1018, 0.1005, 0.1001, 0.0996, 0.1006, 0.0995,
          0.0998], device='cuda:0'),
  tensor([0.0982, 0.0987, 0.1006, 0.1048, 0.1006, 0.1008, 0.0991, 0.1004, 0.0984,
          0.0984], device='cuda:0'),
  tensor([0.0978, 0.0982, 0.0999, 0.1008, 0.1012, 0.1048, 0.0994, 0.1009, 0.0980,
          0.0990], device='cuda:0'),
  tensor([0.0984, 0.0981, 0.1005, 0.1000, 0.1006, 0.1049, 0.0997, 0.0997, 0.0983,
          0.0998], device='cuda:0'),
  tensor([0.0978, 0.0987, 0.1007, 0.1006, 0.1017, 0.1045, 0.0975, 0.1010, 0.0979,
          0.0997], device='cuda:0'),
  tensor([0.0978, 0.0982, 0.1000, 0.1022, 0.0998, 0.1057, 0.1001, 0.0996, 0.0976,
          0.0991], device='cuda:0'),
  tensor([0.0982, 0.0987, 0.0996, 0.1003, 0.1018, 0.1048, 0.0985, 0.1010, 0.0979,
          0.0993], device='cuda:0'),
  tensor([0.0990, 0.0982, 0.1005, 0.0999, 0.1002, 0.1046, 0.1001, 0.1000, 0.0981,
          0.0994], device='cuda:0'),
  tensor([0.0977, 0.0980, 0.1008, 0.1008, 0.1005

In [29]:
similarity.to('cpu').numpy()

array([[0.09920695, 0.09856291, 0.1002874 , 0.10180468, 0.10052148,
        0.10007902, 0.09964986, 0.10059873, 0.09952364, 0.09976529]],
      dtype=float32)

In [5]:
import numpy as np
import torch
torch.tensor([torch.tensor([1,1]),torch.tensor([1,1])])

TypeError: only integer tensors of a single element can be converted to an index

In [28]:
kmeans_with_init(model, testloader,  10, first_centroids/2+centroid_candidate_text.cpu().numpy()/2)

image_ACC 0.8304
image_NMI 0.8290791851615424


(array([[-0.22476196,  0.60941173, -0.2209233 , ...,  0.18920572,
          0.02561769, -0.34466257],
        [ 0.10947472,  0.69288358, -0.13935102, ...,  0.1274251 ,
         -0.09413228, -0.51049289],
        [-0.25009277,  0.64348   ,  0.07371734, ...,  0.16549886,
         -0.03359442, -0.25723271],
        ...,
        [ 0.17551064,  0.82582223,  0.06438381, ...,  0.29377339,
          0.2870351 , -0.47419687],
        [ 0.14123565,  0.44375567,  0.10330949, ..., -0.05486736,
         -0.19089612, -0.42850219],
        [-0.35699216,  0.36745965, -0.03063044, ...,  0.22377988,
         -0.01435056,  0.05094009]]),
 array([0, 6, 6, ..., 0, 1, 8], dtype=int32),
 0.8304)

In [10]:
import numpy as np
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
from torch.autograd import Variable
from sklearn.cluster import KMeans
import torch
from Metrics import nmi, acc, Kmeans_model_evaluation, Kmeans_model_evaluation_text_representation
def kmeans_with_init(model, data_loader, num_cluster, first_centroids):
    model.eval()
    with torch.no_grad():
        dataset_size = len(data_loader.dataset)
        datas = np.zeros([dataset_size, 768])
        label_true = np.zeros(dataset_size)
        ii = 0
        for x, target in data_loader:
            b = x.shape[0]
            x = Variable(x).cuda()
            image_representation = model.encode_image(x)
            u = image_representation.cpu()
            datas[ii * data_loader.batch_size:(ii + 1) *
                  data_loader.batch_size, :] = u.data.numpy()
            label_true[ii * data_loader.batch_size:(ii + 1)
                       * data_loader.batch_size] = target.numpy()
            ii = ii + 1
        kmeans = KMeans(n_clusters=num_cluster,
                        init=first_centroids,max_iter=1,n_init=1).fit(datas)
        label_pred = kmeans.labels_
        centroids = kmeans.cluster_centers_
        ACC = acc(label_true, label_pred, num_cluster)
        print('image_ACC', ACC)
        print('image_NMI', nmi(label_true, label_pred))
        return centroids, label_pred, ACC

In [ ]:
label , acc = kmeans_with_init(model, testloader,  10, centroid_candidate_text.cpu().numpy())

In [23]:
np.mean(np.array(testset.targets) == label)

0.9618

In [13]:
import numpy as np
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
from torch.autograd import Variable
from sklearn.cluster import KMeans
import torch
from Metrics import nmi, acc, Kmeans_model_evaluation, Kmeans_model_evaluation_text_representation, cosine_kmeans_with_init


In [18]:
label , acc = cosine_kmeans_with_init(model.float(), testloader,  10, centroid_candidate_text.float())

image_ACC 0.7633
image_NMI 0.7564347426558758


ValueError: too many values to unpack (expected 2)